# Projet 2 : Estimation du SOC par Intelligence Artificielle - Modèle MLP

Ce notebook implémente un réseau de neurones **Fully Connected (MLP)** pour estimer le State of Charge (SOC) d'une batterie.

## Objectifs
1. Chargement et analyse des données.
2. Prétraitement (Normalisation, Segmentation).
3. Construction et entraînement du modèle MLP.
4. Évaluation des performances (MAE, RMSE, R²).

In [ ]:
# Installation plus robuste des bibliothèques nécessaires
import sys
!{sys.executable} -m pip install tensorflow pandas numpy matplotlib seaborn scikit-learn

print("Installation terminée. SI C'EST LA PREMIÈRE FOIS, VEUILLEZ REDÉMARRER LE NOYAU (KERNEL -> RESTART) avant d'exécuter la suite.")

## 1. Importations et Configuration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math
import os

# Vérification GPU
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## 2. Chargement des Données
Assurez-vous que le fichier `.csv` est accessible.

In [ ]:
# Chemin du fichier (à adapter selon votre environnement)
file_path = 'battery_data_csv_forEstimation.csv'

if os.path.exists(file_path):
    df = pd.read_csv(file_path)
    print(f"Données chargées. Taille: {df.shape}")
else:
    print("Erreur : Fichier non trouvé. Veuillez vérifier le chemin ou uploader le fichier.")

In [ ]:
# Aperçu des données
df.head()

In [ ]:
# Statistiques descriptives
df.describe()

In [ ]:
# Visualisation rapide
plt.figure(figsize=(15, 10))
plt.subplot(4, 1, 1)
plt.plot(df['Current'], label='Current', color='blue')
plt.legend()
plt.subplot(4, 1, 2)
plt.plot(df['Voltage'], label='Voltage', color='orange')
plt.legend()
plt.subplot(4, 1, 3)
plt.plot(df['Temperature'], label='Temperature', color='green')
plt.legend()
plt.subplot(4, 1, 4)
plt.plot(df['SOC'], label='SOC', color='red')
plt.legend()
plt.show()

## 3. Prétraitement pour le MLP
Pour le MLP, nous allons utiliser une fenêtre glissante aplatie.

- **Normalisation** : MinMaxScaler pour mettre toutes les variables entre 0 et 1.
- **Fenêtrage** : Transformation des données en `(samples, window_size * features)`.

In [ ]:
# Sélection des features et de la cible
features = ['Current', 'Voltage', 'Temperature']
target = 'SOC'

data = df[features + [target]].values

# Normalisation
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

print("Données normalisées :", data_scaled.shape)

In [ ]:
def create_dataset_mlp(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        # Features : Colonnes 0, 1, 2 sur la fenêtre temporelle aplatie
        a = dataset[i:(i + look_back), 0:3] 
        X.append(a.flatten()) 
        Y.append(dataset[i + look_back, 3]) # SOC cible
    return np.array(X), np.array(Y)

# Paramètres
LOOK_BACK = 20 # Taille de la fenêtre historique

X, Y = create_dataset_mlp(data_scaled, LOOK_BACK)

print("Shape X :", X.shape)
print("Shape Y :", Y.shape)

In [ ]:
# Split Train/Val/Test (70% / 15% / 15%)
total_size = len(X)
train_size = int(total_size * 0.7)
val_size = int(total_size * 0.15)

X_train, Y_train = X[:train_size], Y[:train_size]
X_val, Y_val = X[train_size:train_size+val_size], Y[train_size:train_size+val_size]
X_test, Y_test = X[train_size+val_size:], Y[train_size+val_size:]

print(f"Train shape: {X_train.shape}")
print(f"Val shape: {X_val.shape}")
print(f"Test shape: {X_test.shape}")

## 4. Construction et Entraînement du Modèle MLP

In [ ]:
def build_mlp_model(input_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid')) # ACTIVATION SIGMOID : Plus précis pour atteindre les min/max (0 et 1) que sigmoid
    
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

model_mlp = build_mlp_model(X_train.shape[1])
model_mlp.summary()

In [ ]:
# Entraînement
history_mlp = model_mlp.fit(
    X_train, Y_train,
    epochs=50, # 50 Époques
    batch_size=64,
    validation_data=(X_val, Y_val),
    verbose=1,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
)

In [ ]:
# Visualisation de la perte (Loss)
plt.figure(figsize=(10, 6))
plt.plot(history_mlp.history['loss'], label='Train Loss')
plt.plot(history_mlp.history['val_loss'], label='Validation Loss')
plt.title('Courbe de Loss (MLP)')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.show()

## 5. Évaluation et Résultats

In [ ]:
# Prédictions sur le jeu de test
predictions_mlp = model_mlp.predict(X_test)

# Métriques
mae = mean_absolute_error(Y_test, predictions_mlp)
rmse = math.sqrt(mean_squared_error(Y_test, predictions_mlp)) # Correction: predictions_mlp ici, pas predictions_lstm
r2 = r2_score(Y_test, predictions_mlp)

print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

In [ ]:
# Tracé SOC réel vs Estimé
plt.figure(figsize=(15, 6))
plt.plot(Y_test, label='SOC Réel', color='black', linewidth=2)
plt.plot(predictions_mlp, label='Estimation MLP', color='red', linestyle='--')
plt.title('SOC Réel vs Estimé (MLP)')
plt.xlabel('Temps (échantillons)')
plt.ylabel('SOC Normalisé')
plt.legend()
plt.show()

In [ ]:
# Erreur d'estimation
error = Y_test - predictions_mlp.flatten()
plt.figure(figsize=(15, 4))
plt.plot(error, color='purple')
plt.title("Erreur d'estimation (Réel - Estimé)")
plt.ylabel('Erreur')
plt.show()

In [ ]:
# Sauvegarde du modèle
model_mlp.save('mlp_soc_model.h5')
print("Modèle MLP sauvegardé.")